In [7]:
!pip install -q datasets
!pip install -q pandas
!pip install -q transformers
!pip install -q tensorflow
!pip install -q huggingface_hub
!pip install -q transformers[torch]
!pip install -q torch
!pip install -q evaluate
!pip install -q jupyterlab
!pip install -q scikit-learn



In [7]:
!pip install -q datasets
!pip install -q pandas
!pip install -q transformers
!pip install -q tensorflow
!pip install -q huggingface_hub
!pip install -q transformers[torch]
!pip install -q torch
!pip install -q evaluate
!pip install -q jupyterlab
!pip install -q scikit-learn



In [7]:
!pip install -q datasets
!pip install -q pandas
!pip install -q transformers
!pip install -q tensorflow
!pip install -q huggingface_hub
!pip install -q transformers[torch]
!pip install -q torch
!pip install -q evaluate
!pip install -q jupyterlab
!pip install -q scikit-learn



In [8]:
!pip install --upgrade accelerate

In [9]:
!pip install -q peft

In [11]:
import os

# Establecer CUDA_VISIBLE_DEVICES en vacío para desactivar la GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [12]:
from datasets import load_dataset

dataset = load_dataset("cbasconc/instructions_Device")
dataset["train"][100]

{'text': 'activar entrada', 'label_text': 'puerta', 'label': 6}

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 811
    })
    test: Dataset({
        features: ['text', 'label_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 202
    })
})

In [15]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(811))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(202))

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=9)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
from transformers import TrainingArguments, Trainer
##falta emter aquí hyper parametros, si no se especifican se usan algunos por defectos, son muchos
training_args = TrainingArguments(output_dir="test_trainer",num_train_epochs = 10, evaluation_strategy="epoch",
                                  per_device_eval_batch_size = 10, per_device_train_batch_size = 10, learning_rate = 1e-5,
                                  weight_decay = 1e-5, use_cpu = True)

In [35]:
import numpy as np
from evaluate import load
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import Trainer, TrainingArguments

# Cargar la métrica de precisión
precision_metric = load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calcular las métricas
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    # Puedes usar la métrica de precisión cargada si lo necesitas para algún propósito específico
    # Si no, simplemente devuelve las métricas calculadas
    return {"precision": precision, "recall": recall, "f1": f1}


In [36]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.086681,0.980794,0.980198,0.977783
2,No log,0.063076,0.986224,0.985149,0.982905
3,No log,0.053911,0.991009,0.990099,0.989800
4,No log,0.043274,0.995310,0.995050,0.995066
5,No log,0.038200,0.995310,0.995050,0.995066
6,No log,0.036555,0.995310,0.995050,0.995066
7,0.033900,0.036399,0.995310,0.995050,0.995066
8,0.033900,0.036160,0.995310,0.995050,0.995066
9,0.033900,0.036270,0.995310,0.995050,0.995066
10,0.033900,0.036706,0.995310,0.995050,0.995066


TrainOutput(global_step=820, training_loss=0.02309485601215828, metrics={'train_runtime': 7047.3593, 'train_samples_per_second': 1.151, 'train_steps_per_second': 0.116, 'total_flos': 2133964770600960.0, 'train_loss': 0.02309485601215828, 'epoch': 10.0})

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.086681,0.980794,0.980198,0.977783
2,No log,0.063076,0.986224,0.985149,0.982905
3,No log,0.053911,0.991009,0.990099,0.989800
4,No log,0.043274,0.995310,0.995050,0.995066
5,No log,0.038200,0.995310,0.995050,0.995066
6,No log,0.036555,0.995310,0.995050,0.995066
7,0.033900,0.036399,0.995310,0.995050,0.995066
8,0.033900,0.036160,0.995310,0.995050,0.995066
9,0.033900,0.036270,0.995310,0.995050,0.995066
10,0.033900,0.036706,0.995310,0.995050,0.995066


TrainOutput(global_step=820, training_loss=0.02309485601215828, metrics={'train_runtime': 7047.3593, 'train_samples_per_second': 1.151, 'train_steps_per_second': 0.116, 'total_flos': 2133964770600960.0, 'train_loss': 0.02309485601215828, 'epoch': 10.0})

In [38]:
model_path = "text-classification-model_local_trainer_V2"
trainer.save_model(model_path)

In [40]:
model.save_pretrained("devices_model_no_trainer_V2")

In [46]:
def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        probs (torch.Tensor): Class probabilities for the input text.
        pred_label_idx (torch.Tensor): The index of the predicted class label.
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    """ Explanation outputs: The BERT model returns a tuple containing the output logits (and possibly other elements depending on the model configuration). In this case, the output logits are the first element in the tuple, which is why we access it using outputs[0].

    outputs[0]: This is a tensor containing the raw output logits for each class. The shape of the tensor is (batch_size, num_classes) where batch_size is the number of input samples (in this case, 1, as we are predicting for a single input text) and num_classes is the number of target classes.

    softmax(1): The softmax function is applied along dimension 1 (the class dimension) to convert the raw logits into class probabilities. Softmax normalizes the logits so that they sum to 1, making them interpretable as probabilities. """

    # Get the index of the class with the highest probability
    # argmax() finds the index of the maximum value in the tensor along a specified dimension.
    # By default, if no dimension is specified, it returns the index of the maximum value in the flattened tensor.
    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label
    # Since pred_label_idx is a tensor containing a single value (the predicted class index),
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [52]:
# Test with a an example text in Turkish
text = "quisiera que porfavor si eres yan amable, abrieras la ventana izquierda, miamor"
# "Machine Learning itself is moving towards more and more automated"
predict(text)

(tensor([[6.3538e-04, 8.6440e-04, 3.6135e-04, 9.8164e-01, 2.7307e-03, 1.0602e-02,
          7.1816e-04, 1.8828e-03, 5.6945e-04]], grad_fn=<SoftmaxBackward0>),
 tensor(3),
 'LABEL_3')

In [51]:
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=9)

In [54]:
import pandas as pd

# Datos de TrainOutput
train_output_data = {
    "global_step": [820],
    "training_loss": [0.02309485601215828],
    "train_runtime": [7047.3593],
    "train_samples_per_second": [1.151],
    "train_steps_per_second": [0.116],
    "total_flos": [2133964770600960.0],
    "train_loss": [0.02309485601215828],
    "epoch": [10.0]
}

# Convertir a DataFrame
train_output_df = pd.DataFrame(train_output_data)

# Guardar como CSV
train_output_df.to_csv('train_output_data.csv', index=False)